# Web Crawler

In [1]:
import requests
from bs4 import BeautifulSoup as soup  # HTML data structure

# extract article hyperlinks from an index page
def extractArtLinks(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text, "html.parser")
    containers = page_soup.findAll("div", {"class": "r-ent"})
    art_links = []
    for container in containers:
        # Finds all link tags "a" from within the first div.
        art_link = container.find('a')
        if art_link:
            #print(art_link['href'])
            #print(container.find('div',{'class':'title'}).get_text())
            art_meta = container.find('div',{'class':'meta'})
            #print(art_meta.find('div',{'class':'author'}).get_text())
            #print(art_meta.find('div',{'class':'date'}).get_text())

            art_links.append({
                'push': container.find('div',{'class':'nrec'}).get_text(),
                'title': container.find('div',{'class':'title'}).get_text().strip(),
                'date': art_meta.find('div',{'class':'date'}).get_text(),
                'author': art_meta.find('div',{'class':'author'}).get_text(),
                'link': art_link['href'],
                'text': extractArtText('https://www.ptt.cc' + art_link['href'])
            })

    return(art_links)

# find the previous index page link
def findPrevIndex(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text,"html.parser")
    btn = page_soup.select('div.btn-group > a')
    up_page_href = btn[3]['href']
    next_page_url = 'https://www.ptt.cc' + up_page_href
    return(next_page_url)

# extract article contents from  the article hyperlink
def extractArtText(url):
    r = requests.get(url, cookies={'over18':'1'})
    page_soup = soup(r.text, "lxml")
    #print(page_soup.find("div",{"id":"main-content"}).get_text())
    art_text=page_soup.select('div#main-content', limit=1)[0].text
    return(art_text)

In [2]:
# main()
num_of_index_page = 2
board_name = 'Food'
url = 'https://www.ptt.cc/bbs/{}/index.html'.format(board_name)
all_links =[]
for page in range(1,num_of_index_page):
    all_links = all_links + extractArtLinks(url)
    url = findPrevIndex(url)
len(all_links)

19

In [3]:
type(all_links[2])
print(all_links[2])

{'push': '1', 'title': '[食記] 新店Oli 不用跑台北就有的浪漫餐酒館', 'date': '10/02', 'author': 'aner', 'link': '/bbs/Food/M.1601622171.A.16A.html', 'text': '作者aner ()看板Food標題[食記] 新店Oli 不用跑台北就有的浪漫餐酒館時間Fri Oct  2 15:02:48 2020\n無音樂圖文網誌版 https://wanwanr9.pixnet.net/blog/post/353015117\n==\n   餐廳名稱：Oli\n   消費時間：2020年/9月\n   地址：新北市新店區中興路三段220巷2號1 樓\n   電話： 02 2910 5887\n==\n\nOh my God  本身住新店那麼久\n\n居然最近是第一次聽到別人跟我介紹一家在大坪林附近的好餐廳\n\n而且還不是新店人跟我推荐的！\n\n常聽到有人在新店粉絲團問  新店那裡可以小酌一杯(不是熱炒店那種)\n\n再加上新店在地  真的是比較多的家庭餐廳或日常小吃店\n\n要找比較有點氣氛  適合約會或安靜談事情的餐廳  真旳是比較少一點\n\n尤其在新店市區\n\n\n這間Oli真的太低調隱密了  低調到在地人都不知道會不會太誇張\n\nOli在中興路三段的小巷子裡  外觀還是一個公寓老房子改造的\n\n我是新店在地人都找了一下  門口也很低調  走過路過真的很容易錯過\n\n不過一進去真的是別有洞天阿 沒想到有這麼有氣氛的地方\n\nOli西班牙餐酒館  主打西班牙料理  也適合喝一杯  氣氛都蠻清輕的\n\n[花生醬乳酪松露薯條180元]  花生醬+乳酪+松露  這三個加起來感覺就是必點啊\n\n本來以為花生醬是會淋在薯條上  結果是沾醬裡有花生醬\n\n保有薯條還是酥酥脆脆充滿松露味  很特別也很好吃\n\n[西班牙烘蛋180元]  本來以為西班牙餐廳的前菜都會像tapas這樣一小碟份量很小\n\n但Oli的份量並不小耶  很蠻有誠意的  光看這烘蛋就非常紥實\n\n蛋香味噴發啊\n\n切開來可以看到非常厚阿\n\n還有一點點的蛋液\n\n上面還有松露醬  也好香  裡面還有馬羚薯塊\n\n\n[伊比利豬老饕上蓋肉 8oz 1280元]  主餐選了伊比

In [4]:
print('Push: {push:s} \n'
      'title: {title:s} \n'
      'date: {date:s} \n'
      'author: {author:s} \n'
      'link: {link:s} \n'
      'text: {text:.5} \n'.format(**all_links[2]))

Push: 1 
title: [食記] 新店Oli 不用跑台北就有的浪漫餐酒館 
date: 10/02 
author: aner 
link: /bbs/Food/M.1601622171.A.16A.html 
text: 作者ane 



:::{admonition} Exercise
How to seperate post texts from push texts?
:::
